In [1]:
# imports
import os
import cv2
from sklearn.ensemble import BaggingClassifier
#import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
#import sklearn as skl
import sklearn.preprocessing as preproc
import numpy as np
import keras as k
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, ZeroPadding2D, Flatten, Dense, Dropout, MaxPool2D
#import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesClassifier
#from time import time
import pandas as pd
from imblearn.over_sampling import SMOTENC
import tensorflow as tf
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator
import math
import shutil
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn.preprocessing import OneHotEncoder

#### variabili globali
#### GIUSEPPE
image_path = '..\skin-cancer-mnist-ham10000\HAM10000_images_part_1'
#### FABIO
# image_path = '..\skin-cancer-mnist-ham10000\HAM10000_images_part_1'
csv_path = 'HAM10000_metadata.csv'
csv_completo = 'dataframe_completo.csv'

Using TensorFlow backend.


In [8]:
dataset = pd.read_csv(csv_completo, encoding = "ISO-8859-1")
batch = 1000

def my_train_batch(dataset, batch, classifier='classifier1.h5'):
    size = len(dataset)
    epoch=0
    cl1 = k.models.load_model('classifier2.h5')
    while size >= batch:
        # To get n random rows 
        samples = dataset.sample(n = batch)
        # splitting test and validation
        X_train, X_val, y_train, y_val = train_test_split(samples["image_id"], samples["dx"], test_size=0.30)
        
        train_img = np.asarray(load_images(X_train))
        val_img = np.asarray(load_images(X_val))
        
        enc = OneHotEncoder(sparse=False)
        y_train_shaped = np.array(y_train).reshape(-1,1)
        y_val_shaped = np.array(y_val).reshape(-1,1)
        enc.fit(y_train_shaped)
        y_train_one = enc.transform(y_train_shaped)
        y_val_one = enc.transform(y_val_shaped)
        
              
        #X_bad_id = X_bad_id.reshape((X_bad_id.shape[0], 1))
        cl1.fit(train_img, y_train_one, epochs=1, shuffle=True, validation_data=(val_img, y_val_one), verbose=2)
        del X_train, y_train_one, X_val, y_val_one, train_img, val_img
        dataset = dataset.drop(samples.index)
        size = len(dataset)
        del samples, enc
        epoch = epoch + 1
        print("fine epoca " + str(epoch) + ";\trestano " + str(size) + "campioni nel dataset")
    cl1.save('classifier2.h5')
    return

# dal dataframe ne estrae le immagini in un array, serve nel batch
def load_images(array):
    images = list()
    #print(array.iloc[0])
    for index in range(0,len(array)):
        #print(index.type())
        img_id = array.iloc[index]
        elem = os.path.join("..\skin-cancer-mnist-ham10000\HAM10000_images_part_1", img_id)
        elem = elem + ".jpg"
        img = cv2.imread(elem)
        # TODO va fatto un reshape, non capisco perché
        images.append(np.asarray(img)) #########################################################################################
    return images


def new_classifier1():
    input_img = Input(shape=(450, 600, 3))  # 3x600x450 image RGB 
    #print (tf.shape(input_img))
    x = Conv2D(20, (5, 5), activation='relu', padding='same')(input_img) # 20x450x600
    #print (tf.shape(x))
    x = MaxPooling2D((2, 2), padding='same')(x) # 20x225x300
    #print (tf.shape(x))
    x = Conv2D(140, (3, 3), activation='relu', padding='same')(x) # 140x75x100
    #print (tf.shape(x))
    x = MaxPooling2D((2, 2), padding='same')(x) # 40x38x50
    #print (tf.shape(x))
    x = Conv2D(50, (3, 3), activation='relu', padding='same')(x) # 50x38x50
    #print (tf.shape(x))
    x = MaxPooling2D((2, 2), padding='same')(x) # 50x19x25
    #print (tf.shape(x))
    x = Flatten()(x) # qui la x diventa monodimensionale
    #print (tf.shape(x))
    # da qualche parte ci va il numero delle label?
    #print(tf.shape(x)) 
    #tf.reshape(x, [1,7])
    #print(tf.shape(x))
    encoded = Dense(7, activation='softmax')(x)
    #print (tf.shape(encoded))
    classifier = k.models.Model(input_img, encoded)   #questo è il nostro base_estimator, compile configura per il training
    classifier.compile(optimizer='adadelta', loss='binary_crossentropy')
    #classifier.compile(optimizer='adadelta', loss='categorical_crossentropy') # <- variante che non risolve il problema del dense
    return classifier

def new_classifier2():
    # Set the CNN model 
    input_shape = (450, 600, 3)
    num_classes = 7
    
    model = k.models.Sequential()
    model.add(Conv2D(20, kernel_size=(5, 5),activation='relu',padding = 'Same', input_shape = input_shape))
    model.add(MaxPool2D(pool_size = (2, 2)))
    model.add(Conv2D(140,kernel_size=(3, 3), activation='relu',padding = 'Same'))
    model.add(MaxPool2D(pool_size = (2, 2)))
    model.add(Conv2D(50,kernel_size=(3, 3), activation='relu',padding = 'Same'))
    model.add(MaxPool2D(pool_size = (2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    #model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    model.summary()
    model.compile(optimizer='adadelta', loss='binary_crossentropy')
    return model

In [9]:
### inizializzazione classificatore sul disco
cl1 = new_classifier1()
cl1.save('classifier2.h5')

In [10]:
samples = dataset.sample(n=1000)
my_train_batch(dataset=samples, batch=50)

Train on 35 samples, validate on 15 samples
Epoch 1/1
 - 10s - loss: 3.6741 - val_loss: 3.2135
fine epoca 1;	restano 950campioni nel dataset
Train on 35 samples, validate on 15 samples
Epoch 1/1
 - 9s - loss: 3.8919 - val_loss: 3.5056
fine epoca 2;	restano 900campioni nel dataset
Train on 35 samples, validate on 15 samples
Epoch 1/1
 - 9s - loss: 3.3897 - val_loss: 3.7978
fine epoca 3;	restano 850campioni nel dataset
Train on 35 samples, validate on 15 samples
Epoch 1/1
 - 10s - loss: 3.7663 - val_loss: 3.2135
fine epoca 4;	restano 800campioni nel dataset
Train on 35 samples, validate on 15 samples
Epoch 1/1
 - 9s - loss: 3.5152 - val_loss: 3.2135
fine epoca 5;	restano 750campioni nel dataset
Train on 35 samples, validate on 15 samples
Epoch 1/1
 - 10s - loss: 3.6408 - val_loss: 3.7978
fine epoca 6;	restano 700campioni nel dataset
Train on 35 samples, validate on 15 samples
Epoch 1/1
 - 10s - loss: 3.8919 - val_loss: 3.7978
fine epoca 7;	restano 650campioni nel dataset
Train on 35 samp

ValueError: Error when checking input: expected input_3 to have 4 dimensions, but got array with shape (35, 1)

In [28]:
prova = dataset.sample(n=100)
t_img = np.asarray(load_images(prova["image_id"]))

(100, 450, 600, 3)

In [33]:
########################LISTE

def my_train_batch_list(dataset, batch, classifier='classifier1.h5'):
    size = len(dataset)
    epoch=0
    while size >= batch:
        # To get n random rows 
        samples = dataset.sample(n = batch)
        # splitting test and validation
        X_train, X_test1, y_train, y_test1 = train_test_split(samples["image_id"], samples["dx"], test_size=0.30)
        X_test, X_val, y_test, y_val = train_test_split(X_test1, y_test1, test_size=0.40)
        del X_test1, y_test1
        
        train_img = load_images(X_train)
        test_img = load_images(X_test)  # NB non sono mai usate, pensare se servano oppure no
        val_img = load_images(X_val)
        
        
        #X_bad_id = X_bad_id.reshape((X_bad_id.shape[0], 1))
        cl1 = k.models.load_model('classifier2.h5')
        cl1.fit(train_img, y_train, epochs=1, shuffle=True, validation_data=(val_img, y_val), verbose=2)
        cl1.save('classifier1.h5')
        del X_train, y_train, X_test, y_test, X_val, y_val, train_img, test_img, val_img, classifier
        dataset = dataset.drop(samples.index)
        size = len(dataset)
        del samples
        epoch = epoch + 1
        print("fine epoca " + epoch + ";\trestano " + size + "campioni nel dataset")
    return


def load_images_list(array):
    images = list()
    #print(array.iloc[0])
    for index in range(0,len(array)):
        #print(index.type())
        img_id = array.iloc[index]
        elem = os.path.join("..\skin-cancer-mnist-ham10000\HAM10000_images_part_1", img_id)
        elem = elem + ".jpg"
        img = cv2.imread(elem)
        # TODO va fatto un reshape, non capisco perché
        images.append(img) #########################################################################################
    return images

In [34]:
samples = dataset.sample(n=5000)
my_train_batch(dataset=samples, batch=batch)

ValueError: Error when checking target: expected dense_8 to have shape (7,) but got array with shape (1,)